In [64]:
from stats import run_algo
from typing import List
from tqdm.notebook import tqdm
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib_inline.backend_inline import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

In [65]:
import os
filename = 'result2.txt'

def read_expanded(filename=filename):
    with open(filename, 'r') as path_file:
        hl_expanded, ll_expanded = list(map(int, path_file.read().split('\n')[0].split(' ')[1:]))
        return hl_expanded, ll_expanded

def run(alg_name, map_name, scen_name, num_agents, num_repetitions, w=None):
    hl_expanded = 0
    ll_expanded = 0
    count_done = 0
    for i in tqdm(range(num_repetitions)):
        done = run_algo(alg=alg_name, dest_file=filename,
                        map_path=os.path.join(os.curdir, 'data', 'maps', 'mapf', map_name),
                        scen_path=os.path.join(os.curdir, 'data', 'scens', 'mapf', scen_name),
                        tasks_count=num_agents,
                        test_num=i, w=w)
        count_done += int(done)
        if done:
            hl, ll = read_expanded(filename)
            hl_expanded += hl * int(done)
            ll_expanded += ll * int(done)
    if count_done == 0:
        return (0, 0, 0)
    return count_done / num_repetitions, hl_expanded / count_done, ll_expanded / count_done

In [66]:
from collections import defaultdict
def parse_path_file(path_file_content):
    agents_path = defaultdict(list)
    agent = None
    lines = path_file_content.split('\n')[1:]
    for line in lines:
        if line == '':
            break
        nums = list(map(int, line.split(' ')))
        if len(nums) == 1:
            agent = int(nums[0])
        else:
            x, y, time = nums
            agents_path[agent].append((x, y, time))
    return agents_path

def read_paths(filename=filename):
    with open(filename, 'r') as path_file:
        path_file_content = path_file.read()
        agents_path = parse_path_file(path_file_content)
    return dict(agents_path)

In [67]:
def gen_rate_list(alg:str, map_name:str, scen_name:str, actors_array:List[int], N=20, w=None):
    rate_list = []
    for num_actors in actors_array:
        rate, hl_expanded, ll_expanded = run(alg, map_name, scen_name, num_actors, N, w)
        print(f'Num actors: {num_actors}, Rate: {rate:.2f}, High level expanded: {hl_expanded:.2f}, Low level expanded: {ll_expanded:.2f}')
        rate_list.append((rate, hl_expanded, ll_expanded))
        if rate <= 0.1:
            break
    while len(rate_list) < len (actors_array):
        rate_list.append((0.0, 0, 0))
    
    return rate_list

In [68]:
rand_actors = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90]

In [69]:
# CBS random-32x32x10
rand_cbs = gen_rate_list('CBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen', rand_actors)
rand_cbs

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 121.75, Low level expanded: 185341.80


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 166.05, Low level expanded: 292156.05


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 60, Rate: 0.20, High level expanded: 14670.50, Low level expanded: 42565758.50


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 70, Rate: 0.00, High level expanded: 0.00, Low level expanded: 0.00


[(1.0, 121.75, 185341.8),
 (1.0, 166.05, 292156.05),
 (1.0, 397.15, 806727.9),
 (0.7, 320.0, 781256.9285714285),
 (0.55, 6494.090909090909, 12333912.545454545),
 (0.3, 1746.3333333333333, 4101592.1666666665),
 (0.2, 14670.5, 42565758.5),
 (0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0)]

In [70]:
# ECBS random-32x32x10 w=1.5 more agents
rand_ecbs15 = gen_rate_list('ECBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen',rand_actors, w=1.5)
rand_ecbs15

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.25, Low level expanded: 2297.70


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 1.65, Low level expanded: 7136.75


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 80, Rate: 0.80, High level expanded: 491.06, Low level expanded: 4723048.06


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 1.00, High level expanded: 1001.00, Low level expanded: 9456378.00


[(1.0, 1.25, 2297.7),
 (1.0, 1.65, 7136.75),
 (1.0, 4.1, 27739.7),
 (1.0, 7.95, 58830.45),
 (1.0, 18.4, 151392.8),
 (1.0, 31.7, 282567.25),
 (1.0, 104.85, 836182.2),
 (1.0, 316.05, 3066169.0),
 (0.8, 491.0625, 4723048.0625),
 (1.0, 1001.0, 9456378.0)]

In [71]:
# ECBS random-32x32x10 w=1.3 more agents
rand_ecbs13 = gen_rate_list('ECBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen', rand_actors, w=1.3)
rand_ecbs13

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.25, Low level expanded: 1972.50


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 1.65, Low level expanded: 5822.15


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 80, Rate: 0.80, High level expanded: 491.06, Low level expanded: 3006200.50


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 1.00, High level expanded: 1001.00, Low level expanded: 6131710.00


[(1.0, 1.25, 1972.5),
 (1.0, 1.65, 5822.15),
 (1.0, 4.1, 21517.85),
 (1.0, 7.95, 43414.25),
 (1.0, 18.4, 107535.3),
 (1.0, 31.7, 194503.35),
 (1.0, 104.85, 565433.4),
 (1.0, 312.9, 2073554.3),
 (0.8, 491.0625, 3006200.5),
 (1.0, 1001.0, 6131710.0)]

In [72]:
# ECBS random-32x32x10 w=1.1 more agents
rand_ecbs11 = gen_rate_list('ECBS', 'random-32-32-10.map', 'random-32-32-10-even-1.scen', rand_actors, w=1.1)
rand_ecbs11

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.25, Low level expanded: 1633.55


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 1.00, High level expanded: 1.65, Low level expanded: 4394.40


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 80, Rate: 0.75, High level expanded: 489.60, Low level expanded: 1372743.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 90, Rate: 0.00, High level expanded: 0.00, Low level expanded: 0.00


[(1.0, 1.25, 1633.55),
 (1.0, 1.65, 4394.4),
 (1.0, 4.1, 13608.2),
 (1.0, 8.7, 26420.35),
 (1.0, 26.6, 92643.1),
 (1.0, 50.05, 142877.8),
 (1.0, 110.8, 298624.8),
 (1.0, 390.35, 1461673.35),
 (0.75, 489.6, 1372743.0),
 (0, 0, 0)]

In [73]:
brc_actors = [1, 3, 5, 7, 10, 15, 20, 25, 30]

In [74]:
# CBS brc202d
brc_cbs = gen_rate_list('CBS', 'brc202d.map', 'brc202d-even-1.scen', brc_actors)
brc_cbs

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 1, Rate: 1.00, High level expanded: 1.00, Low level expanded: 20799.25


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 3, Rate: 1.00, High level expanded: 1.00, Low level expanded: 61720.05


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 0.65, High level expanded: 1.85, Low level expanded: 9210007.08


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 15, Rate: 0.10, High level expanded: 1.50, Low level expanded: 1506442.50


[(1.0, 1.0, 20799.25),
 (1.0, 1.0, 61720.05),
 (1.0, 1.35, 3342086.65),
 (0.95, 1.736842105263158, 6446621.052631579),
 (0.65, 1.8461538461538463, 9210007.076923076),
 (0.1, 1.5, 1506442.5),
 (0.0, 0, 0),
 (0.0, 0, 0),
 (0.0, 0, 0)]

In [75]:
# ECBS brc202d w=1.5
brc_ecbs15 = gen_rate_list('ECBS', 'brc202d.map', 'brc202d-even-1.scen', brc_actors, w=1.5)
brc_ecbs15

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 1, Rate: 1.00, High level expanded: 1.00, Low level expanded: 20799.25


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 3, Rate: 1.00, High level expanded: 1.00, Low level expanded: 61720.05


  0%|          | 0/20 [00:00<?, ?it/s]

...

Num actors: 7, Rate: 0.50, High level expanded: 1.10, Low level expanded: 972761.80


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 10, Rate: 0.20, High level expanded: 1.25, Low level expanded: 2287587.00


  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [18]:
# ECBS brc202d w=1.3
brc_ecbs13 = gen_rate_list('ECBS', 'brc202d.map', 'brc202d-even-1.scen', brc_actors, w=1.3)
brc_ecbs13

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :1, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :3, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :7, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :10, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


[1.0, 1.0, 1.0, 1.0, 1.0]

In [19]:
# ECBS brc202d w=1.1
brc_ecbs11 = gen_rate_list('ECBS', 'brc202d.map', 'brc202d-even-1.scen', brc_actors, w=1.1)
brc_ecbs11

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :1, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :3, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :7, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :10, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


[1.0, 1.0, 1.0, 1.0, 1.0]

In [43]:
den_actors = [1, 3, 5, 7, 10, 15, 20, 25, 30]

In [44]:
# CBS den520d 
den_cbs = gen_rate_list('CBS', 'den520d.map', 'den520d-even-1.scen', den_actors)
den_cbs

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 1, Rate: 1.00, High level expanded: 1.00, Low level expanded: 8485.35


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 5, Rate: 1.00, High level expanded: 1.05, Low level expanded: 118503.75


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 35, Rate: 0.25, High level expanded: 10.00, Low level expanded: 8825907.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors: 40, Rate: 0.05, High level expanded: 7.00, Low level expanded: 7915208.00


[(1.0, 1.0, 8485.35),
 (1.0, 1.05, 118503.75),
 (1.0, 1.35, 748904.2),
 (0.95, 2.5789473684210527, 2440920.4210526315),
 (0.85, 5.0588235294117645, 5133373.705882353),
 (0.7, 6.357142857142857, 6594147.357142857),
 (0.45, 7.666666666666667, 8540223.666666666),
 (0.25, 10.0, 8825907.0),
 (0.05, 7.0, 7915208.0)]

In [21]:
# ECBS den520d w=1.5
den_ecbs15 = gen_rate_list('ECBS', 'den520d.map', 'den520d-even-1.scen', den_actors, w=1.5)
den_ecbs15

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :5, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :7, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :66, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :70, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [22]:
# ECBS den520d w=1.3
den_ecbs13 = gen_rate_list('ECBS', 'den520d.map', 'den520d-even-1.scen', den_actors, w=1.3)
den_ecbs13

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :5, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :7, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :66, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :70, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [23]:
# ECBS den520d w=1.1
den_ecbs11 = gen_rate_list('ECBS', 'den520d.map', 'den520d-even-1.scen', den_actors, w=1.1)
den_ecbs11

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :5, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :7, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :66, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


  0%|          | 0/20 [00:00<?, ?it/s]

Num actors :70, Rate: 1.00, High level expanded: 440.00, Low level expanded: 1155369.00


[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]